In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from dataset.gaijin_double_g import *

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# --- 运行数据生成 ---
GRID_SIZE = 32
NUM_PROJECTIONS = 3 # 假设我们使用3个投影角度
NUM_SAMPLES = 10  # 训练样本数

print("正在生成数据集...")
# 
# 图示：三维体素网格中的火焰模型以及从不同角度观察到的二维投影图像。
X_train, Y_train = create_dataset(NUM_SAMPLES, GRID_SIZE, NUM_PROJECTIONS,use_random_angles=True)
print(f"输入形状 (2D 投影): {X_train.shape}")
print(f"输出形状 (3D 模型): {Y_train.shape}")
    # 可视化3D模型和投影
    # 可视化3D模型和投影
print("\nVisualizing 3D model and projections...")

fig1=visualize_3d_gaussian(Y_train[5],title="3D Gaussian Distribution",threshold=0.1)
fig2 = visualize_projections(X_train[1], f"{NUM_PROJECTIONS} Projections")


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from dataset.gaijin_double_g import *
from c_net_1 import*

# --- 运行数据生成 ---
GRID_SIZE = 32
NUM_PROJECTIONS = 3 # 假设我们使用3个投影角度
NUM_SAMPLES = 1000  # 训练样本数

print("正在生成数据集...")
# 
# 图示：三维体素网格中的火焰模型以及从不同角度观察到的二维投影图像。
X_train, Y_train = create_dataset(NUM_SAMPLES, GRID_SIZE, NUM_PROJECTIONS,use_random_angles=True)
print(f"输入形状 (2D 投影): {X_train.shape}")
print(f"输出形状 (3D 模型): {Y_train.shape}")

# 创建 PyTorch DataLoader
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# --- 实例化模型 ---
model = Flame3DReconstructionNet(input_channels=NUM_PROJECTIONS, output_size=GRID_SIZE).to(device)


# 损失函数: 均方误差 (MSE)
criterion = nn.MSELoss()
# 优化器: Adam
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --- 训练参数 ---
num_epochs = 50

print("开始训练模型...")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, targets in train_loader:
        # targets 形状: (B, D, H, W)
        # inputs 形状: (B, C, H, W)
        
        # 将数据移到 GPU/CPU
        inputs, targets = inputs.to(device), targets.to(device)
        
        # 梯度清零
        optimizer.zero_grad()
        
        # 前向传播
        outputs = model(inputs)
        
        # 计算损失
        loss = criterion(outputs, targets)
        
        # 反向传播和优化
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.6f}')

print("训练完成！")

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch


# 假设模型已经训练完成，并且我们有了 X_train, Y_train, model, device 等变量

def visualize_reconstruction(model, X_data, Y_data, sample_index, slice_dim=0, slice_idx=None):
    """
    可视化真实模型和重建模型的一个切片。
    
    Args:
        model (nn.Module): 训练好的模型。
        X_data (Tensor): 输入的 2D 投影数据集。
        Y_data (Tensor): 真实的 3D 模型数据集。
        sample_index (int): 要可视化的样本索引。
        slice_dim (int): 切片维度 (0=X, 1=Y, 2=Z)。
        slice_idx (int): 切片索引 (例如，如果 size=32,取 16 为中心切片)。
    """
    
    model.eval() # 切换到评估模式
    
    # 提取输入和真实目标
    input_projections = X_data[sample_index:sample_index+1].to(device)
    true_3d = Y_data[sample_index].cpu().numpy()
    
    # 获取重建结果
    with torch.no_grad():
        reconstructed_3d = model(input_projections).squeeze(0).cpu().numpy()

    size = true_3d.shape[0]
    
    # 确定切片索引，默认取中心切片
    if slice_idx is None:
        slice_idx = size // 2
    
    # 提取切片
    if slice_dim == 0:
        true_slice = true_3d[slice_idx, :, :]
        reconstructed_slice = reconstructed_3d[slice_idx, :, :]
        dim_label = 'X'
    elif slice_dim == 1:
        true_slice = true_3d[:, slice_idx, :]
        reconstructed_slice = reconstructed_3d[:, slice_idx, :]
        dim_label = 'Y'
    else: # slice_dim == 2
        true_slice = true_3d[:, :, slice_idx]
        reconstructed_slice = reconstructed_3d[:, :, slice_idx]
        dim_label = 'Z'

    # --- 绘图 ---
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # 1. 真实模型切片
    im1 = axes[0].imshow(true_slice, cmap='hot', origin='lower')
    axes[0].set_title(f'Ground Truth (True Flame Model) - {dim_label}-Slice {slice_idx}')
    axes[0].axis('off')
    fig.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04)

    # 2. 重建模型切片
    im2 = axes[1].imshow(reconstructed_slice, cmap='hot', origin='lower')
    axes[1].set_title(f'Reconstruction (CNN Output) - {dim_label}-Slice {slice_idx}')
    axes[1].axis('off')
    fig.colorbar(im2, ax=axes[1], fraction=0.046, pad=0.04)
    
    plt.tight_layout()
    plt.show()

# --- 运行可视化 ---

# 假设我们在训练集中选取第一个样本进行可视化
SAMPLE_TO_VISUALIZE = 1

# 可视化 Z 轴的中心切片 (slice_dim=2)
print("\n--- 可视化 Z 轴中心切片 ---")
visualize_reconstruction(model, X_train, Y_train, 
                         sample_index=SAMPLE_TO_VISUALIZE, 
                         slice_dim=2, 
                         slice_idx=GRID_SIZE // 2)

# 可视化 X 轴的中心切片 (slice_dim=0)
print("\n--- 可视化 X 轴中心切片 ---")
visualize_reconstruction(model, X_train, Y_train, 
                         sample_index=SAMPLE_TO_VISUALIZE, 
                         slice_dim=0, 
                         slice_idx=GRID_SIZE // 2)